In [1]:
!pip install transformers bitsandbytes sentencepiece accelerate guidance --upgrade -qq

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
!pip install --upgrade transformers -qq
!pip install accelerate
!pip install -q -U google-generativeai

In [4]:
!wget -nc https://people.eecs.berkeley.edu/~hendrycks/data.tar

--2024-06-06 11:40:13--  https://people.eecs.berkeley.edu/~hendrycks/data.tar
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166184960 (158M) [application/x-tar]
Saving to: 'data.tar'

data.tar            100%[===================>] 158.49M  55.9MB/s    in 2.8s    

2024-06-06 11:40:16 (55.9 MB/s) - 'data.tar' saved [166184960/166184960]



In [5]:
import tarfile
unzip_path = '.'
tar = tarfile.open('data.tar')
tar.extractall(path=unzip_path)
tar.close()

In [6]:
!pip install --upgrade transformers -qq
!pip install accelerate
!pip install groq
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 1.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 864.7 kB/s eta 0:00:00--:--


In [7]:
import torch
import numpy as np
from sklearn.isotonic import IsotonicRegression
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, pipeline
from transformers import BitsAndBytesConfig
import torch.nn.functional as F
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import google.generativeai as genai
from bert_score import score
import os
from groq import Groq
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import json
import re
import concurrent.futures
from tqdm import tqdm
from yaml import safe_load

2024-06-06 11:41:15.136296: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 11:41:15.136400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 11:41:15.300479: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# # Define models and tokenizers
# model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
# tokenizer_mistral = AutoTokenizer.from_pretrained(model_name)
# tokenizer_mistral.pad_token = tokenizer_mistral.eos_token

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )

# mistral_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16,
#     quantization_config=bnb_config,
#     low_cpu_mem_usage=True,
#     device_map="auto",
# )

In [10]:
# model_name2 = 'stabilityai/StableBeluga-13B'
# tokenizer_beluga = AutoTokenizer.from_pretrained(model_name2)
# tokenizer_beluga.pad_token = tokenizer_beluga.eos_token

# beluga_model = AutoModelForCausalLM.from_pretrained(
#     model_name2,
#     torch_dtype=torch.float16,
#     quantization_config=bnb_config,
#     low_cpu_mem_usage=True,
#     device_map="auto",
# )

In [11]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(filtered_tokens)

In [12]:
# Utility functions
def load_questions(file_path):
    questions = []
    with open(file_path, 'r') as f:
        for line in f:
            questions.append(json.loads(line.strip()))
    return questions

def load_model_answers(dir_path):
    model_answers = {}
    for file_name in os.listdir(dir_path):
        model_name = file_name.split('.')[0]
        model_answers[model_name] = {}
        with open(os.path.join(dir_path, file_name), 'r') as f:
            for line in f:
                answer = json.loads(line.strip())
                question_id = answer['question_id']
                model_answers[model_name][question_id] = answer
    return model_answers

def get_endpoint(endpoint_config):
    # Implementation depends on the format of your API config file
    return endpoint_config

def make_config(file_path):
    with open(file_path, 'r') as f:
        return safe_load(f)

def chat_completion_groq(model, conv, temperature, max_tokens, api_key):
    client = Groq(api_key=api_key)
    chat_completion = client.chat.completions.create(
        messages=conv,
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return chat_completion.choices[0].message.content

def get_score(judgment, pattern, pairwise=True):
    matches = pattern.findall(judgment)
    matches = [m for m in matches if m != ""]
    if len(set(matches)) == 0:
        return None, True
    elif len(set(matches)) == 1:
        if pairwise:
            return matches[0].strip("\n"), False
        return int(matches[0])
    else:
        return None, False

In [13]:
client = Groq(api_key="YOUR_API")

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the meaning of life? How modern Ai is affeting the real life and what will be the Ai outcomes in future?",
        }

    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

**What is the meaning of life?**

A question that has puzzled philosophers, theologians, and thinkers for centuries! The meaning of life is a complex and subjective concept that has been debated and explored in various ways. Here are some possible perspectives:

1. **Existentialism**: Life has no inherent meaning; we create our own purpose and meaning through our experiences, choices, and relationships.
2. **Religious or spiritual**: Life's meaning is derived from a higher power, divine plan, or spiritual pursuit, providing a sense of purpose and direction.
3. **Humanism**: Life's meaning is found in human connections, personal growth, and the pursuit of happiness, freedom, and fulfillment.
4. **Biological**: Life's meaning is simply to survive, reproduce, and ensure the continuation of the species.
5. **Philosophical**: Life's meaning is a quest for knowledge, wisdom, and self-awareness, often involving the pursuit of truth, beauty, and goodness.

Ultimately, the meaning of life is a 

### Pipeline

In [15]:
class EvaluationPipeline:
    def __init__(self, api_key, judge_model, regex_pattern, temperature=0.8, max_tokens=300, pairwise=True):
        self.api_key = api_key
        self.judge_model = judge_model
        self.regex_pattern = re.compile(regex_pattern)
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.pairwise = pairwise
        self.prompts = []
        self.responses = []
        self.references = []
        self.confidences = []
        self.accuracies = []

    def get_mistral_answer(self, prompt, include_stopwords=True):
        inputs = tokenizer_mistral.encode_plus(
            prompt, return_tensors='pt', max_length=1024, truncation=True, padding='max_length'
        ).to(mistral_model.device)

        outputs = mistral_model.generate(
            inputs['input_ids'], attention_mask=inputs['attention_mask'],
            max_new_tokens=300, num_return_sequences=1, temperature=0.8,
            do_sample=True, output_scores=True, return_dict_in_generate=True
        )

        response_ids = outputs.sequences[0]
        response_text = tokenizer_mistral.decode(response_ids, skip_special_tokens=True)

        logits = outputs.scores  # Logits of the generated tokens
        softmax_probs = torch.softmax(torch.stack(logits), dim=-1)

        tokens = tokenizer_mistral.convert_ids_to_tokens(response_ids)
        if include_stopwords:
            token_confidences = [prob.max().item() for prob in softmax_probs]
        else:
            token_confidences = [prob.max().item() for prob, token in zip(softmax_probs, tokens) if token.lower() not in stop_words]

        mean_confidence = np.mean(token_confidences)

        return response_text, mean_confidence

    def judge_answer(self, response, prompt):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"""
                        Review the user’s question and the corresponding response using the binary scoring system described below.
                        - 0 points: The response is incorrect or does not address the user’s question.
                        - 1 point: The response is correct and addresses the user’s question.

                        User: {prompt}
                        Response: {response}
                        """
                }
            ],
            model="llama3-70b-8192",
        )

        llama3_response = chat_completion.choices[0].message.content.strip()
        return llama3_response

    def parse_evaluation(self, evaluation):
        if "1 point" in evaluation:
            return 1
        return 0

    def generate_reference_text(self, prompt):
        inputs = tokenizer_mistral.encode_plus(
            prompt, return_tensors='pt', max_length=1024, truncation=True, padding='max_length'
        ).to(mistral_model.device)

        outputs = mistral_model.generate(
            inputs['input_ids'], attention_mask=inputs['attention_mask'],
            max_new_tokens=300, num_return_sequences=1, temperature=0.8, do_sample=True
        )

        reference_text = tokenizer_mistral.decode(outputs[0], skip_special_tokens=True)
        return reference_text

    def calculate_bertscore(self, references, candidates):
        P, R, F1 = score(candidates, references, lang="en", model_type="bert-base-uncased")
        return P.mean().item(), R.mean().item(), F1.mean().item()

    def evaluate(self, prompts):
        self.prompts = prompts  # Ensure prompts are stored
        for prompt in prompts:
            response, confidence = self.get_mistral_answer(prompt)
            self.responses.append(response)
            self.confidences.append(confidence)
            reference_text = self.generate_reference_text(prompt)
            self.references.append(reference_text)
            judgement = self.judge_answer(response, prompt)
            accuracy = self.parse_evaluation(judgement)
            self.accuracies.append(accuracy)

        precision, recall, f1 = self.calculate_bertscore(self.references, self.responses)
        print(f"BERTScore - Precision: {precision}, Recall: {recall}, F1: {f1}")

    def calculate_ece(self):
        # Ensure all lists have the same length
        if len(self.prompts) == len(self.responses) == len(self.confidences) == len(self.accuracies):
            data = pd.DataFrame({
                'prompt': self.prompts,
                'response': self.responses,
                'confidence': self.confidences,
                'rating': self.accuracies
            })
        else:
            raise ValueError("All arrays must be of the same length")

        # Normalize confidence scores
        data['confidence_normalized'] = data['confidence'] / data['confidence'].max()

        # Bin the normalized confidence scores
        bins = np.linspace(0, 1, 11)
        data['bin'] = pd.cut(data['confidence_normalized'], bins=bins, labels=False, include_lowest=True)

        # Calculate accuracy for each bin
        bin_accuracies = data.groupby('bin')['rating'].mean()
        bin_proportions = data['bin'].value_counts(normalize=True)

        # Drop bins with NaN values
        valid_bins = bin_accuracies.dropna().index
        bin_accuracies = bin_accuracies[valid_bins]
        bin_proportions = bin_proportions[valid_bins]
        bin_confidences = (bins[:-1] + bins[1:]) / 2
        bin_confidences = bin_confidences[valid_bins]

        # Ensure lengths match
        bin_confidences = bin_confidences[:len(bin_accuracies)]

        # Compute ECE
        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
        print(f"Expected Calibration Error (ECE): {ece}")

        return data, bins, bin_accuracies

    def plot_reliability_diagram(self, bins, bin_accuracies):
        plt.figure(figsize=(10, 6))
        plt.plot((bins[:-1] + bins[1:]) / 2, bin_accuracies, marker='o', label='Accuracy per bin')
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect calibration')
        plt.xlabel('Confidence')
        plt.ylabel('Accuracy')
        plt.title('Reliability Diagram')
        plt.legend()
        plt.grid(True)
        plt.show()

In [16]:
pipeline = EvaluationPipeline(api_key="Your_api_key", judge_model="llama3-70b-8192", regex_pattern=r"(correct|incorrect)")
prompts = ["What is nuclear fusion?", "Tell me about the current AI situation in the world?", "How does a combustion engine work?"]
pipeline.evaluate(prompts)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTScore - Precision: 0.6630645990371704, Recall: 0.6872342228889465, F1: 0.6748167872428894


In [20]:
data, bins, bin_accuracies = pipeline.calculate_ece()

Expected Calibration Error (ECE): 0.050000000000000044


In [19]:
few_shot_examples = """
Q: What is nuclear fusion?
A: Nuclear fusion is a reaction in which two atomic nuclei combine to form a heavier nucleus, releasing energy in the process.

Q: How does a combustion engine work?
A: A combustion engine works by burning fuel in a combustion chamber to produce mechanical energy that drives the engine.
"""

def get_mistral_answer_few_shot(prompt):
    few_shot_prompt = few_shot_examples + f"\nQ: {prompt}\nA:"
    response, confidence = get_mistral_answer(few_shot_prompt)
    return response, confidence

# Example usage:
few_shot_response, few_shot_confidence = get_mistral_answer_few_shot("Tell me about the current AI situation in the world.")
print(f"Few-shot response: {few_shot_response}")
print(f"Few-shot confidence: {few_shot_confidence}")

### Class pipeline for mmlu dataset

In [17]:
class Tokenizer:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def encode(self, text, max_length=1024):
        return self.tokenizer.encode_plus(text, return_tensors='pt', max_length=max_length, truncation=True)

    def decode(self, tokens):
        return self.tokenizer.decode(tokens, skip_special_tokens=True)

In [18]:
class EvaluationPipeline:
    def __init__(self, api_key, judge_model_name, smaller_model_name, temperature=0.8, max_tokens=300):
        self.api_key = api_key
        self.judge_model_name = judge_model_name
        self.temperature = temperature
        self.max_tokens = max_tokens


        self.tokenizer = Tokenizer(smaller_model_name)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )

        self.main_model = AutoModelForCausalLM.from_pretrained(
            smaller_model_name,
            torch_dtype=torch.float16,
            quantization_config=bnb_config,
            low_cpu_mem_usage=True,
            device_map="auto",
        )

        self.categories = {
            "abstract_algebra": ["math"],
            "anatomy": ["health"],
            "astronomy": ["physics"],
            "business_ethics": ["business"],
            "clinical_knowledge": ["health"],
            "college_biology": ["biology"],
            "college_chemistry": ["chemistry"],
            "college_computer_science": ["computer science"],
            "college_mathematics": ["math"],
            "college_medicine": ["health"],
            "college_physics": ["physics"],
            "computer_security": ["computer science"],
            "conceptual_physics": ["physics"],
            "econometrics": ["economics"],
            "electrical_engineering": ["engineering"],
            "elementary_mathematics": ["math"],
            "formal_logic": ["philosophy"],
            "global_facts": ["other"],
            "high_school_biology": ["biology"],
            "high_school_chemistry": ["chemistry"],
            "high_school_computer_science": ["computer science"],
            "high_school_european_history": ["history"],
            "high_school_geography": ["geography"],
            "high_school_government_and_politics": ["politics"],
            "high_school_macroeconomics": ["economics"],
            "high_school_mathematics": ["math"],
            "high_school_microeconomics": ["economics"],
            "high_school_physics": ["physics"],
            "high_school_psychology": ["psychology"],
            "high_school_statistics": ["math"],
            "high_school_us_history": ["history"],
            "high_school_world_history": ["history"],
            "human_aging": ["health"],
            "human_sexuality": ["culture"],
            "international_law": ["law"],
            "jurisprudence": ["law"],
            "logical_fallacies": ["philosophy"],
            "machine_learning": ["computer science"],
            "management": ["business"],
            "marketing": ["business"],
            "medical_genetics": ["health"],
            "miscellaneous": ["other"],
            "moral_disputes": ["philosophy"],
            "moral_scenarios": ["philosophy"],
            "nutrition": ["health"],
            "philosophy": ["philosophy"],
            "prehistory": ["history"],
            "professional_accounting": ["other"],
            "professional_law": ["law"],
            "professional_medicine": ["health"],
            "professional_psychology": ["psychology"],
            "public_relations": ["politics"],
            "security_studies": ["politics"],
            "sociology": ["culture"],
            "us_foreign_policy": ["politics"],
            "virology": ["health"],
            "world_religions": ["philosophy"],
        }

    def get_model_answer(self, prompt):
        """Generates an answer from the specified model."""
        inputs = self.tokenizer.encode(prompt).to(self.main_model.device)
        outputs = self.main_model.generate(
            inputs['input_ids'], 
            max_length=self.max_tokens, 
            num_return_sequences=1, 
            temperature=self.temperature, 
            output_scores=True, 
            return_dict_in_generate=True
        )
        response = self.tokenizer.decode(outputs.sequences[0])

        # Get the confidence score
        logits = torch.stack(outputs.scores, dim=1)
        probs = F.softmax(logits, dim=-1)

        token_ids = outputs.sequences[:, inputs['input_ids'].shape[1]:]
        confidences = probs.gather(2, token_ids.unsqueeze(-1)).squeeze(-1).mean(dim=1).detach().cpu().numpy()

        avg_confidence = confidences[0]
        return response, avg_confidence

    def judge_answer(self, response, prompt):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"""
                        Review the user’s question and the corresponding response using the binary scoring system described below.
                        - 0 points: The response is incorrect or does not address the user’s question.
                        - 1 point: The response is correct and addresses the user’s question.

                        User: {prompt}
                        Response: {response}
                        """
                }
            ],
            model=self.judge_model_name,
        )

        judge_response = chat_completion.choices[0].message.content.strip()
        return judge_response

    def parse_evaluation(self, evaluation):
        return 1 if "1 point" in evaluation else 0

    def generate_reference_text(self, prompt):
        inputs = self.tokenizer.encode(prompt).to(self.main_model.device)
        outputs = self.main_model.generate(
            inputs['input_ids'], 
            attention_mask=inputs['attention_mask'],
            max_new_tokens=self.max_tokens, 
            num_return_sequences=1, 
            temperature=self.temperature, 
            do_sample=True
        )
        reference_text = self.tokenizer.decode(outputs[0])
        return reference_text

    def calculate_bertscore(self, references, candidates):
        P, R, F1 = score(candidates, references, lang="en", model_type="bert-base-uncased")
        return P.mean().item(), R.mean().item(), F1.mean().item()

    def evaluate_from_csv(self, csv_file_path):
        # Clear data before processing each file
        self.prompts = []
        self.responses = []
        self.references = []
        self.confidences = []
        self.accuracies = []

        df = pd.read_csv(csv_file_path)
        self.prompts = df.iloc[:, 0].tolist()  # Take only the first column (questions)

        for prompt in self.prompts:
            response, confidence = self.get_model_answer(prompt)
            self.responses.append(response)
            self.confidences.append(confidence)
            reference_text = self.generate_reference_text(prompt)
            self.references.append(reference_text)
            judgement = self.judge_answer(response, prompt)
            accuracy = self.parse_evaluation(judgement)
            self.accuracies.append(accuracy)

        precision, recall, f1 = self.calculate_bertscore(self.references, self.responses)
        print(f"BERTScore - Precision: {precision}, Recall: {recall}, F1: {f1}")

        data, bins, bin_accuracies = self.calculate_ece()
        data.to_csv("results.csv", index=False)  # Save results to CSV
        return data, bins, bin_accuracies

    def evaluate_folder(self, folder_path):
        results = {}  # Store ECE values for each class
        category_results = {category: [] for category in set(cat for sublist in self.categories.values() for cat in sublist)}

        for filename in os.listdir(folder_path):
            if filename.endswith(".csv"):
                filepath = os.path.join(folder_path, filename)
                class_name = filename[:-8]  # Extract class name from filename without "_dev.csv"
                
                category_name = None
                for key, value in self.categories.items():
                    if key == class_name:
                        category_name = value[0]
                        break

                if category_name:
                    print(f"Evaluating {class_name} in category {category_name}...")
                    data, bins, bin_accuracies = self.evaluate_from_csv(filepath)

                    # Align shapes and calculate ECE
                    bin_confidences = (bins[:-1] + bins[1:]) / 2  # Confidences for each bin
                    valid_bins = bin_accuracies.dropna().index  # Bins with data
                    bin_accuracies = bin_accuracies[valid_bins]
                    bin_proportions = data['bin'].value_counts(normalize=True)[valid_bins]  # Get proportions for valid bins
                    bin_confidences = bin_confidences[valid_bins]  # Select confidences for valid bins

                    ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
                    category_results[category_name].append(ece)
                    print(f"{class_name} ECE: {ece}")

        # Calculate average ECE for each category and save to CSV
        average_ece_results = {category: np.mean(ece_list) for category, ece_list in category_results.items()}
        average_ece_df = pd.DataFrame({'Category': average_ece_results.keys(), 'Average ECE': average_ece_results.values()})
        average_ece_df.to_csv("category_results.csv", index=False)

    def calculate_ece(self):
        # Ensure all lists have the same length
        if len(self.prompts) == len(self.responses) == len(self.confidences) == len(self.accuracies):
            data = pd.DataFrame({
                'prompt': self.prompts,
                'response': self.responses,
                'confidence': self.confidences,
                'rating': self.accuracies
            })
        else:
            raise ValueError("All arrays must be of the same length")

        # Normalize confidence scores
        data['confidence_normalized'] = data['confidence'] / data['confidence'].max()

        # Bin the normalized confidence scores
        bins = np.linspace(0, 1, 11)
        data['bin'] = pd.cut(data['confidence_normalized'], bins=bins, labels=False, include_lowest=True)

        # Calculate accuracy for each bin
        bin_accuracies = data.groupby('bin')['rating'].mean()
        bin_proportions = data['bin'].value_counts(normalize=True)

        # Drop bins with NaN values
        valid_bins = bin_accuracies.dropna().index
        bin_accuracies = bin_accuracies[valid_bins]
        bin_proportions = bin_proportions[valid_bins]
        bin_confidences = (bins[:-1] + bins[1:]) / 2
        bin_confidences = bin_confidences[valid_bins]

        # Ensure lengths match
        bin_confidences = bin_confidences[:len(bin_accuracies)]

        # Compute ECE
        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
        print(f"Expected Calibration Error (ECE): {ece}")

        return data, bins, bin_accuracies

In [ ]:
pipeline = EvaluationPipeline(api_key="Your_api_key", judge_model_name="llama3-70b-8192", smaller_model_name="mistralai/Mistral-7B-Instruct-v0.2")
pipeline.evaluate_folder("/kaggle/working/data/dev")

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


BERTScore - Precision: 0.7571039199829102, Recall: 0.7485461235046387, F1: 0.7515882849693298
Expected Calibration Error (ECE): 0.19999999999999996
high_school_psychology ECE: 0.19999999999999996
Evaluating computer_security in category computer science...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end

BERTScore - Precision: 0.6872113943099976, Recall: 0.6256282329559326, F1: 0.653347909450531
Expected Calibration Error (ECE): 0.07500000000000001
computer_security ECE: 0.07500000000000001
Evaluating moral_scenarios in category philosophy...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
